In [1]:
import pandas as pd
from Bio.Blast import NCBIWWW
from Bio.Blast.Applications import NcbiblastxCommandline

In [94]:
import pandas as pd

bovine_ms_aa= pd.read_csv("MS_Data_with_percent_homology.csv")
bovine_ms_aa.head(10)

,Unnamed: 0,Unique,Total,reference,Gene Symbol,MWT(kDa),AVG,Bovine_Seq,%identity_with_chlammy
0,0,133,154,tr|A0A3Q1NE82|A0A3Q1NE82_BOVIN,AHNAK,614.26,3.1225,MEKEEETTRELLLPNWQGSGSHGLTITQRDDGVFVQEVMQNSPAAR...,0.000
1,1,7,109,sp|Q3MHM5|TBB4B_BOVIN,TUBB4B,49.80,3.3008,MREIVHLQAGQCGNQIGAKFWEVISDEHGIDPTGTYHGDSDLQLER...,90.465
2,2,13,101,sp|Q2T9S0|TBB3_BOVIN,TUBB3,50.40,3.3130,MREIVHIQAGQCGNQIGAKFWEVISDEHGIDPSGNYVGDSDLQLER...,85.581
3,3,62,98,tr|A0A3Q1M0Y8|A0A3Q1M0Y8_BOVIN,CROCC2,169.52,3.1375,MDLEDALGRLEAAEQRSTSLSQVNALLREQLEQLRKAHDRLAEELA...,0.000
4,4,16,91,tr|F6RP72|F6RP72_BOVIN,LOC100295712; tubulin alpha chain,50.85,3.0554,MPHSLLCFQRECISIHVGQAGVQIGNACWELYCLEHGIQPDGQMPS...,87.097
5,5,80,88,tr|F1MMQ8|F1MMQ8_BOVIN,SPTA1,280.78,3.3721,MESDGPEVLETAEEIQARRQEVLSRYQRFKERVAERGQKLEDSYHY...,0.000
6,6,71,84,tr|F1MKE9|F1MKE9_BOVIN,SPTB,268.25,3.5225,MTSATEFENVGNQPPYSRINARWDGPDDELDNDNSSARLFERSRIK...,0.000
7,7,18,83,tr|A0A452DIL8|A0A452DIL8_BOVIN,TUBB2B,49.94,3.7333,MREIVHIQAGQCGNQIGAKFWEVISDEHGIDPTGSYHGDSDLQLER...,89.767
8,8,27,75,tr|Q58DT9|Q58DT9_BOVIN,ACTA2,45.19,2.7932,MCEEEDSTALVCDNGSGLCKAGFAGDDAPRAVFPSIVGRPRHQGVM...,87.870
9,9,68,72,tr|A0A3Q1MRS3|A0A3Q1MRS3_BOVIN,PLEC,535.22,2.9450,MVAGMFMPLDQLRAIYEVLFREGVMVAKKDRRPRSLHPHVPGVTNL...,49.533


In [95]:
chlammy_seq_match=[]

for n in range(bovine_ms_aa.shape[0]):
    with open("test.txt",'w') as f:
        f.write(bovine_ms_aa["Bovine_Seq"][n])

    blastx_cline = NcbiblastxCommandline(cmd="blastp",query="test.txt", db="Chlammy_seqs.fasta", evalue=0.001,outfmt=6)
    J=blastx_cline()
    J=J[0].split("\n")
    try:
        chlammy_seq_match.append(J[0].split("\t")[1])
    except:
        chlammy_seq_match.append(0)

In [96]:
bovine_ms_aa['ChlammySeqName']=chlammy_seq_match

Apparently when I saved the dataframe as a csv I did not use the argument to not include an index. Ill drop that here.

In [97]:
bovine_ms_aa=bovine_ms_aa.drop(["Unnamed: 0"],axis=1)

Some functions to programatically access the Uniprot database

In [98]:
import urllib.parse
import urllib.request

def req_uniport(gene):
    #Specify the database to query - we are just quering the general protein database
    url = 'https://www.uniprot.org/uniprot/'

    #entering the parameters we care about
    params= {
        'query':gene,
        'format':'fasta'
    }

    data=urllib.parse.urlencode(params)
    data = data.encode('utf-8')
    req = urllib.request.Request(url,data)
    
    return req

def decode_uniport(req):
    with urllib.request.urlopen(req) as f:
        response = f.read()
        response = response.decode('utf-8')
    
    return response

def from_response_get_aaonly(response):
    start = int(0)

    for n_line, i in enumerate(response.split()):
        if i[:2] == "SV":
            start=n_line
            break
    return "".join(response.split()[n_line+1:])

    
def get_uniprot_aa_seq(gene):
    request=req_uniport(gene)
    response=decode_uniport(request)
    seq=from_response_get_aaonly(response)
    return seq
    

In [99]:
gene_names=[]
for k,i in enumerate(bovine_ms_aa['ChlammySeqName']):
    c=0
    if i == 0:
        gene_names.append(0)
        continue
    fasta=decode_uniport(req_uniport(i))
    for block in fasta.split():
        if "GN=" in block:
            gene_names.append(block[3:])
            c=1
            break
    
    if c==0:
        try:
            gene_names.append("_".join(fasta.split()[0].split("|")[1:3]))
        except:
            gene_names.append("Seq not in Uniprot")
        

Just want to change around the columns

In [100]:
bovine_ms_aa['Chlammy_GeneName']=gene_names

In [104]:
bovine_ms_aa.columns.tolist()

['Unique',
 'Total',
 'reference',
 'Gene Symbol',
 'MWT(kDa)',
 'AVG',
 'Bovine_Seq',
 '%identity_with_chlammy',
 'ChlammySeqName',
 'Chlammy_GeneName']

In [107]:
new_order=['Unique',
 'Total',
 'reference',
 'Gene Symbol',
 'Chlammy_GeneName',
 'MWT(kDa)',
 'AVG',
 'Bovine_Seq',
 '%identity_with_chlammy',
 'ChlammySeqName']

In [110]:
bovine_ms_aa=bovine_ms_aa[new_order]

In [ ]:
bovine_ms_aa.to_csv("MS_HomologyANDChlammyName.csv",index=False)

Apparently some sequences are not reconized in the Uniprot database. This is an example where a sequence was found, but the uniprot did not have the sequence. Perhaps I could "unbreak" this by not just dumbly taking the top sequence hit from the blast?

In [82]:
bovine_ms_aa.loc[[121]]

,Unique,Total,reference,Gene Symbol,MWT(kDa),AVG,Bovine_Seq,%identity_with_chlammy,ChlammySeqName
121,9,11,sp|P68432|H31_BOVIN,Histone H3.1,15.39,2.2491,MARTKQTARKSTGGKAPRKQLATKAARKSAPATGGVKKPHRYRPGT...,95.588,S59581
